In [1]:
#!/usr/bin/env python
"""Convert a GFF and associated FASTA file into GenBank format.

Usage:
    gff_to_genbank.py <GFF annotation file> [<FASTA sequence file> <molecule type>]

 FASTA sequence file: input sequences matching records in GFF. Optional if sequences
   are in the GFF
 molecule type: type of molecule in the GFF file. Defaults to DNA, the most common case.
"""
from __future__ import print_function

import sys
import os

from Bio import SeqIO

from BCBio import GFF

import logging
log_format = '%(levelname)-8s %(asctime)s   %(message)s'
date_format = "%d/%m %H:%M:%S"
logging.basicConfig(format=log_format, datefmt=date_format, level=logging.DEBUG)



def main(gff_file, fasta_file, out_file, molecule_type="DNA"):
    #out_file = "%s.gb" % os.path.splitext(gff_file)[0]
    if fasta_file:
        fasta_input = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
    else:
        fasta_input = {}
    gff_iter = GFF.parse(gff_file, fasta_input)
    return gff_iter
    #SeqIO.write(_check_gff(_fix_ncbi_id(gff_iter), molecule_type), out_file, "genbank")


def _fix_ncbi_id(fasta_iter):
    """GenBank identifiers can only be 16 characters; try to shorten NCBI.
    """
    for rec in fasta_iter:
        if len(rec.name) > 16 and rec.name.find("|") > 0:
            new_id = [x for x in rec.name.split("|") if x][-1]
            print("Warning: shortening NCBI name %s to %s" % (rec.id, new_id))
            rec.id = new_id
            rec.name = new_id
        yield rec


def _check_gff(gff_iterator, molecule_type):
    """Check GFF files before feeding to SeqIO to be sure they have sequences.
    """
    for rec in gff_iterator:
        if "molecule_type" not in rec.annotations:
            rec.annotations["molecule_type"] = molecule_type
        yield _flatten_features(rec)


def _flatten_features(rec):
    """Make sub_features in an input rec flat for output.

    GenBank does not handle nested features, so we want to make
    everything top level.
    """
    out = []
    for f in rec.features:
        cur = [f]
        while len(cur) > 0:
            nextf = []
            for curf in cur:
                out.append(curf)
                if len(curf.sub_features) > 0:
                    nextf.extend(curf.sub_features)
            cur = nextf
    rec.features = out
    return rec

In [2]:

gff_in = "../data/interim/prokka/1097422/1097422.gff"
fna_in = "../data/interim/fasta/1097422.fna"
output = "test/1097422.gff"
output_gbk = "test/1097422.gbk"
output_faa = "test/097422.faa"
gff_iter = main(gff_in, fna_in, output_gbk)

In [3]:
gff_iter = _check_gff(_fix_ncbi_id(gff_iter), "DNA")

In [4]:
gff = [i for i in gff_iter]

In [5]:
gff[0].id

'scaffold_1'

In [6]:
GBcds = gff
all_entries = []
for cds in GBcds:
        if cds.seq is not None:
            cds.id = cds.name
            cds.description = ''
            all_entries.append(cds)

In [7]:
for i in all_entries[0].features:
    if i.type == "CDS":
        test = i

In [8]:
test.translate(all_entries[0].seq, cds=False)

Seq('AKSHAAKKGKAKEPSEEGEEDCDIVKEEVTSDAVA*')

In [10]:
test

SeqFeature(FeatureLocation(ExactPosition(3137975), ExactPosition(3138083), strand=1), type='CDS', id='CDS_1114')

In [11]:
#"1097422"
for i in all_entries[0].features:
    if i.id == "gene_6":
        print(i)
        print(i.translate(all_entries[0].seq))

type: gene
location: [53217:55265](+)
id: gene_6
qualifiers:
    Key: ID, Value: ['gene_6']
    Key: Name, Value: ['jgi.p|Aspdef1|28']
    Key: portal_id, Value: ['Aspdef1']
    Key: product_name, Value: ['hypothetical protein']
    Key: proteinId, Value: ['28']
    Key: score, Value: ['0']
    Key: source, Value: ['prediction']
    Key: transcriptId, Value: ['216']

LLTDSVDERKPLFCYILPSLRYRLSHCTPHTSTHSPRSAISPICISTCKFTSKPQ*S*RRQSLRSILMRCTNAPKKAAISSFSVFHSNQSTVSFHLKFLKSPLCESQKTVV*VGAGSINCHS*SISAFPRCSNGHRRLLTNCLLYGAAIYAWSVFVLVPSGNEAESSNTFLEERNPGVSKSDKTNHTTHEKQRVANTDAIFIPLGWPQLRQGELYKESDPEWKAFVKISKDREKIKALKGNTYYRIILFLY*YSHE*MSWHHLF*LRRLDLMPSRACWDLLSPYHSIG*YPFFHPVHHPHIIVLGM*RFLLCNLCAPHDDIG*RLPRLEYRGAQDLCRIRTPILFTSLYDLLLLPSP*RMPT*YSGGVL*IGSTRIAQTMSAH*VWRARQLKHCCRRIPNN*ASSTKRHGLNHRILPLPDPGAVPVKLTETHTLTLQ*FCPYYSGFLCPMLAPGLIFTRPR*LSNRD*MHVELVDQMYVTETHSLSVVLWALGDPWGPAGLRYRANMTLKPQGGCLFPCVLRMGRYSDEILLAVNRAALPNPAATNTVSAHD*L*M*SFTRKPVVPGTSLFPIKEWG*LLEACRIGMLSNFPRSVKNPGDKFWRPESDLRCVWCWLARSTLKTEAGAPPF

/datadrive/home/miniconda3/envs/antismash_env/lib/python3.8/site-packages/Bio/Seq.py:2334: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [62]:
fasta = "../data/interim/fasta/1097422.fna"
gff="../data/interim/prokka/1097422/1097422.gff"

In [60]:
from antismash.common import gff_parser
from antismash.common import record_processing

def jgi_gff_to_gbk(fasta, taxon, gff):
    records = record_processing.parse_input_sequence(fasta, 
                                                     taxon=taxon,
                                                     gff_file=gff)
    records_biopython = [r.to_biopython() for r in records]
    return records_biopython

In [64]:
records = jgi_gff_to_gbk(fasta, "fungi", gff)

INFO     17/08 22:25:41   Parsing input sequence '../data/interim/fasta/1097422.fna'
DEBUG    17/08 22:25:42   Loading annotations from GFF file
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_1
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_2
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_3
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_4
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_5
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_6
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_7
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_8
DEBUG    17/08 22:25:48   Stripping antiSMASH features and annotations from record: scaffold_9


In [77]:
for record in records:
    print(record.id)
    for feature in record.features:
        print(feature.qualifiers['gene'], feature.qualifiers['translation'])

scaffold_1
['jgi.p_Aspdef1_53449'] ['MAQLSDQRVHGTKRPAEGELDDQPLAKRFGRLRIGPLAAALLSQNREDGARLEPELNRFNDPMMLDDTKSTVYIHDLEHELAENEVLDGTFKILPGLDDRLSISRMLVTSSKLHCNELVLYREPESLSIPKDKDLVRRALIETRERARLRHLNPRHTVEEEPRSLGPNQSDGTRNSNHHQQLYDDLMEVDTELGS']
['jgi.p_Aspdef1_53451'] ['MTSKKEDGRNWTSKAVRDTFLQYFRNNGHTFVASSPVAPHSDPTLLFTNAGMNQFKSIFLGTVNPNSDFAQLKRAVNSQKCIRAGGKHNDLDDVGKDSYHHTFFEMLGNWSFGDYFKKEAIGYSWELLTQVYGLDPSRLYVTYFEGNEKGGLEPDLDARELWRAVGVPEDHILPGNMKDNFWEMGDQGPCGPCSEIHYDRIGGRNAASLVNQDDPNVLEIWNNVFIQYNRETDGSLRPLPNKHVDTGMGFERLVSVLQDKSSNYDTDVFSPLFATIQNITGARDYQGRFGSDDSDGIDTAYRVVADHVRTLSFAISDGVVPNNEGRGYVIRRVLRRGARYARKYFNVEIGNFFSKIVPTLVEQLGDMFPELRHKEQDVKEILNEEEISFAKTLDRGERQFEQYAQQAKIAGVRKLHGADVWRLYDTFGFPVDLTRIMADERGLLIDDREFEDARAKAKEASKGQKQTTVGAVKLDVHDLGKLEKMNGVPKTDDSAKFEKGNITAQIKAIYQGKAFHSSTQGLLDSEQIGVILDRTNFYAEQGGQENDAGRIIIDGQAELEVVDVQLYAGYVLHTGFMKYGAFSVDDTVICEYDELRRWPIRNNHTGTHILNFALREVLGDGVEQKGSLVSPEKLRFDFSHKSPITDAELAKIEDLSTQYIRRNCLVYAQDVPLAIARRISGVRAVFGETYPDPVRVVSVGAELSEILKNVEDPRWEEVSIEFCGGTHAQKTGDIKDLII

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [31]:
for record in records:
    if any(feature.type == "CDS" for feature in record.features):
        continue
    record.features.extend(gff_features.get(record.id, [])) 

In [37]:
records = record_processing.parse_input_sequence("../data/interim/fasta/1097422.fna",
                                      gff_file="../data/interim/prokka/1097422/1097422.gff")

INFO     17/08 22:06:31   Parsing input sequence '../data/interim/fasta/1097422.fna'
DEBUG    17/08 22:06:32   Loading annotations from GFF file
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_1
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_2
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_3
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_4
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_5
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_6
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_7
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_8
DEBUG    17/08 22:06:40   Stripping antiSMASH features and annotations from record: scaffold_9


In [54]:
a = records[0].to_biopython()

In [59]:
print(a.features[1])

type: CDS
location: join{[44892:44986](+), [45066:45212](+), [45259:45286](+), [45337:45373](+), [45432:47841](+), [47911:48085](+)}
qualifiers:
    Key: ID, Value: ['CDS_2']
    Key: gene, Value: ['jgi.p_Aspdef1_53451']
    Key: source, Value: ['prediction']
    Key: transl_table, Value: ['11']
    Key: translation, Value: ['MTSKKEDGRNWTSKAVRDTFLQYFRNNGHTFVASSPVAPHSDPTLLFTNAGMNQFKSIFLGTVNPNSDFAQLKRAVNSQKCIRAGGKHNDLDDVGKDSYHHTFFEMLGNWSFGDYFKKEAIGYSWELLTQVYGLDPSRLYVTYFEGNEKGGLEPDLDARELWRAVGVPEDHILPGNMKDNFWEMGDQGPCGPCSEIHYDRIGGRNAASLVNQDDPNVLEIWNNVFIQYNRETDGSLRPLPNKHVDTGMGFERLVSVLQDKSSNYDTDVFSPLFATIQNITGARDYQGRFGSDDSDGIDTAYRVVADHVRTLSFAISDGVVPNNEGRGYVIRRVLRRGARYARKYFNVEIGNFFSKIVPTLVEQLGDMFPELRHKEQDVKEILNEEEISFAKTLDRGERQFEQYAQQAKIAGVRKLHGADVWRLYDTFGFPVDLTRIMADERGLLIDDREFEDARAKAKEASKGQKQTTVGAVKLDVHDLGKLEKMNGVPKTDDSAKFEKGNITAQIKAIYQGKAFHSSTQGLLDSEQIGVILDRTNFYAEQGGQENDAGRIIIDGQAELEVVDVQLYAGYVLHTGFMKYGAFSVDDTVICEYDELRRWPIRNNHTGTHILNFALREVLGDGVEQKGSLVSPEKLRFDFSHKSPITDAELAKIEDLSTQYIRRNCLVYAQDVP